In [118]:
import asyncio
import os
import random

from utils_experiment import run_reorder_task
from utils_metrics import lcs_length, bubble_sort_distance, combined_normalized_loss
%reload_ext autoreload
%autoreload 2
%autoreload now

import logging

from semaphore import set_semaphore
from utils_openai import call_gpt4

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

from dotenv import load_dotenv
from openai import AsyncOpenAI

load_dotenv()

True

In [83]:
set_semaphore("gpt-4", 100)

In [84]:
client = AsyncOpenAI()

In [86]:
comics = os.listdir("data/xkcd")

In [122]:
comics_sample = random.sample(comics, 10)

In [123]:
results = await asyncio.gather(
    *[
        run_reorder_task(
            client=client,
            panels_path=f"data/xkcd/{comic}",
            # model_name="gpt-4o",
            model_name="gpt-4o-mini",
        ) for comic in comics_sample
    ] 
)

2024-09-16 19:49:11,281 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-16 19:49:12,253 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-16 19:49:12,406 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-16 19:49:12,647 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-16 19:49:12,743 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-16 19:49:13,627 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-16 19:49:13,659 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-16 19:49:13,850 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-16 19:49:14,714 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "

In [124]:
metrics = [combined_normalized_loss(result["correct_order"], result["predicted_order"]) for result in results]

In [125]:
avg_metric = sum(metrics) / len(metrics)

In [126]:
avg_metric

0.3991666666666666

In [127]:
results[9]

{'panels_path': 'data/xkcd/1090-FormalLanguages.jpg',
 'imgs_original': ['panel_1.jpg', 'panel_0.jpg', 'panel_2.jpg'],
 'imgs_predicted': ['panel_0.jpg', 'panel_1.jpg', 'panel_2.jpg'],
 'correct_order': [1, 0, 2],
 'predicted_order': [1, 0, 2],
 'reasoning': '1. In the first panel, a stick figure enthusiastically shouts "GRAMMAR!" while another figure looks on, suggesting a focus on language or grammar-related content.  \n2. The second panel shows a chaotic scene where the same stick figure crashes into the setting of a symposium, indicating a sudden disruption.  \n3. The third panel depicts the second figure still at the podium, seemingly unfazed, while the first figure runs away, suggesting a humorous or absurd reaction to the situation.  \n\nThe overall narrative seems to revolve around a grammar enthusiast who disrupts a formal event (the symposium) with their excitement about grammar. The first panel sets the stage for the enthusiasm, the second panel introduces chaos with the cra

In [100]:
result = await run_reorder_task(
    client=client,
    # panels_path="data/xkcd/13-Canyon.jpeg",
    # panels_path="data/xkcd/69-PillowTalk.jpeg",
    panels_path="data/xkcd/73-Zeppelin.jpeg",
    model_name="gpt-4o-mini",
)

2024-09-16 19:42:51,059 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [101]:
result

{'panels_path': 'data/xkcd/73-Zeppelin.jpeg',
 'imgs_original': ['panel_2.jpg', 'panel_1.jpg', 'panel_3.jpg', 'panel_0.jpg'],
 'imgs_predicted': ['panel_2.jpg',
  'panel_0.jpg',
  'panel_1.jpg',
  'panel_3.jpg'],
 'correct_order': [3, 1, 0, 2],
 'predicted_order': [0, 3, 1, 2],
 'reasoning': '1. In the first panel, two stick figures are walking together, suggesting a casual conversation or outing.  \n2. The second panel shows a close-up of a watch with "Zeppelin!" written on it, indicating a specific time or event related to a zeppelin.  \n3. The third panel depicts a zeppelin in the sky, with the two stick figures looking up at it, suggesting they are excited or surprised by its presence.  \n4. The fourth panel features one stick figure asking the other, "What time is it?" which implies they are checking the time in relation to the zeppelin.  \n\nThe overall narrative seems to follow two friends who are out together when they notice a zeppelin in the sky. They start by walking and cha

In [102]:
print(result['reasoning'])

1. In the first panel, two stick figures are walking together, suggesting a casual conversation or outing.  
2. The second panel shows a close-up of a watch with "Zeppelin!" written on it, indicating a specific time or event related to a zeppelin.  
3. The third panel depicts a zeppelin in the sky, with the two stick figures looking up at it, suggesting they are excited or surprised by its presence.  
4. The fourth panel features one stick figure asking the other, "What time is it?" which implies they are checking the time in relation to the zeppelin.  

The overall narrative seems to follow two friends who are out together when they notice a zeppelin in the sky. They start by walking and chatting (panel 1), then one of them checks their watch, which has a special note about the zeppelin (panel 2). This leads to their curiosity about the time (panel 4), just before they spot the zeppelin overhead (panel 3). The sequence flows from their interaction to the discovery of the zeppelin, cre

In [108]:
combined_normalized_loss(
    result["correct_order"],
    result["predicted_order"],
)

0.29166666666666663

In [69]:
bubble_sort_distance(
    [5, 0, 3, 1, 2, 6, 4],
    [5, 4, 6, 0, 3, 1, 2],
)

9

In [80]:
combined_normalized_loss(
    [5, 0, 3, 1, 2, 6, 4],
    [5, 0, 3, 1, 2, 4, 6],
)

0.09523809523809526

In [77]:
lcs_length(
    [5, 0, 3, 1, 2, 6, 4],
    [5, 0, 3, 1, 2, 6, 4],
)

7

In [8]:
# TODO - think of metric
# https://chatgpt.com/c/66e1daab-cce4-8008-abb2-65fb209c3328

In [4]:
response = await call_gpt4(
    client,
    "Explain what is in those pictures",
    imgs_paths=[
        "data/xkcd/13-Canyon.jpeg/panel_0.jpg",
        "data/xkcd/13-Canyon.jpeg/panel_1.jpg",
        "data/xkcd/13-Canyon.jpeg/panel_2.jpg",
        "data/xkcd/13-Canyon.jpeg/panel_3.jpg",
        "data/xkcd/13-Canyon.jpeg/panel_4.jpg",
        "data/xkcd/13-Canyon.jpeg/panel_5.jpg",
        "data/xkcd/13-Canyon.jpeg/panel_6.jpg",
    ]
)

2024-09-11 22:49:34,885 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [7]:
response

'These images appear to be a series of comic strips featuring stick figures, likely from a webcomic. Here\'s a brief explanation of each:\n\n1. **First Image**: One stick figure asks another, "What time is it?" This sets up a conversation about time.\n\n2. **Second Image**: The second stick figure points to a line, possibly representing time or a timeline, indicating a moment or event.\n\n3. **Third Image**: The first stick figure responds with "Now," emphasizing the present moment in relation to time.\n\n4. **Fourth Image**: A more abstract scene shows a vast landscape with a canyon and celestial bodies in the sky, possibly representing a metaphorical or philosophical concept about time or existence.\n\n5. **Fifth Image**: The first stick figure comments that the previous answer was "pretty boring," suggesting a desire for a more engaging discussion.\n\n6. **Sixth Image**: The second stick figure simply replies, "Is not," indicating a playful disagreement.\n\n7. **Seventh Image**: The